<a href="https://colab.research.google.com/github/karamthodishh/Music-Generator/blob/main/RNN_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from music21 import converter, instrument, note

In [2]:
file_path = '/content/maestro-v3.0.0-midi (2).zip/2015/'

def load_data(file_path):
    music = converter.parse(file_path)

    # Extract the notes
    note_sequence = []
    for note in music.flat.notes:
        if isinstance(note, chord.Chord):
            for pitch in note.pitches:
                note_sequence.append(pitch.midi)
        else:
            note_sequence.append(note.pitch.midi)

    return note_sequence



In [3]:
def save_data(note_sequence, file_path):
    # Create a music21 stream
    music_stream = music21.stream.Stream()

    # Create a note object for each pitch
    for pitch in note_sequence:
        note = music21.note.Note(midi=pitch)
        music_stream.append(note)

    # Write the stream to a MIDI file
    music_stream.write('midi', fp=file_path)

In [4]:
def preprocess_data(note_sequence, sequence_length):
    # Create a list of input sequences and target notes
    sequences = []
    notes = []

    # For each note in the sequence
    for i in range(len(note_sequence) - sequence_length):
        sequences.append(note_sequence[i:i + sequence_length])
        notes.append(note_sequence[i + sequence_length])

    # Pad the sequences to the maximum length
    sequences = np.array(sequences)
    notes = np.array(notes)

    return sequences, notes

In [5]:
def create_model(input_shape, num_notes):
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dropout(0.3))
    model.add(Dense(num_notes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy')

    return model

In [6]:
def train_model(model, sequences, notes, epochs=200, batch_size=64):
    # Create the training data
    training_data = sequences[:-1]
    target_notes = notes[:-1]

    # Train the model
    model.fit(training_data, target_notes, epochs=epochs, batch_size=batch_size)

In [7]:
def generate_music(model, seed_sequence, sequence_length, num_notes):
    generated_sequence = list(seed_sequence)

    for _ in range(num_steps):
        # Prepare the input sequence
        input_sequence = np.array(seed_sequence[-sequence_length:]).reshape(1, -1)

        # Predict the next note
        predicted_note = model.predict(input_sequence, verbose=0)

        # Sample the next note using the predicted probabilities
        next_note = np.random.choice(num_notes, p=predicted_note[0])

        # Append the new note to the generated sequence
        generated_sequence.append(next_note)

    return generated_sequence

In [8]:
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.9 MB/s eta 0:00:00


In [9]:
!pip install Chord

In [10]:
import mido
import numpy as np

class Chord:
    def __init__(self, notes, name, duration):
        self.notes = notes
        self.name = name
        self.duration = duration

def preprocess_data(note_sequence, sequence_length):
    sequences = np.zeros((len(note_sequence) // sequence_length, sequence_length, 2))
    notes = np.zeros((len(note_sequence) // sequence_length, sequence_length, 2))

    for i in range(0, len(note_sequence), sequence_length):
        sequence_end = i + sequence_length

        if sequence_end > len(note_sequence):
            sequence_end = len(note_sequence)

        sequences[i // sequence_length, :sequence_end - i] = [[note.time, note.data] for note in note_sequence[i:sequence_end] if isinstance(note, mido.Message) and note.type == 'note_on']
        notes[i // sequence_length, :sequence_end - i] = [[note.time, note.data] for note in note_sequence[i:sequence_end] if isinstance(note, mido.Message) and note.type == 'note_on']

    return sequences, notes

In [11]:
def generate_chords(note_sequence):
    chords = []
    current_chord = None

    for note in note_sequence:
        if isinstance(note, mido.Message) and note.type == 'note_on':
            if current_chord is None:
                current_chord = Chord([note.note], "major", note.time)
            elif current_chord.duration == note.time:
                current_chord.notes.append(note.note)
            else:
                chords.append(current_chord)
                current_chord = Chord([note.note], "major", note.time)
        elif isinstance(note, mido.Message) and note.type == 'note_off':
            if current_chord is not None:
                current_chord.duration = note.time
                chords.append(current_chord)
                current_chord = None

    return chords

In [12]:
def pitch_to_pc(pitch):
    octave = pitch // 12 - 1
    note = pitch % 12
    return note + 12 * octave

In [13]:
import mido

def generate_music_file(chords, filename):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    for chord in chords:
        for note in chord.notes:
            track.append(mido.Message('note_on', note=pc_to_pitch(note), velocity=127, time=chord.start_time))
            track.append(mido.Message('note_off', note=pc_to_pitch(note), velocity=127, time=chord.end_time))

    mid.save(filename)

In [14]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592284 sha256=44590bfd1546e7ef60255aa918e2b9d1086140e0dbee9f029ee43c3268bd1f7b
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [15]:
!pip install Note

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 23.5 MB/s eta 0:00:00
  Created wheel for Note: filename=note-0.5.2-py2.py3-none-any.whl size=27083 sha256=4df14f73e50bcc47d7c043e1dd5c2d3ca76e7f3ba1f028fc2e196b0bc866c11f
  Stored in directory: /root/.cache/pip/wheels/57/2b/fb/8b503b1708d77aa7b13c1e9ac70bdb4dc5c8bace191b678753
Successfully built Note


In [16]:
pip install --upgrade pretty_midi

In [17]:
def notes_to_chord(notes, duration):
  """Converts a list of notes and durations to a PrettyMIDI Chord object.

  Args:
    notes: A list of notes in the range [0, 127].
    duration: The duration of the chord in beats.

  Returns:
    A PrettyMIDI Chord object.
  """

  chord = pm.Chord(notes, duration)
  return chord

In [18]:
note_to_midi = {
    'C0': 12, 'D0': 14, 'E0': 16, 'F0': 17, 'G0': 19, 'A0': 21, 'B0': 23,
    'C1': 24, 'D1': 26, 'E1': 28, 'F1': 29, 'G1': 31, 'A1': 33, 'B1': 35,
    'C2': 36, 'D2': 38, 'E2': 40, 'F2': 41, 'G2': 43, 'A2': 45, 'B2': 47,
    'C3': 48, 'D3': 50, 'E3': 52, 'F3': 53, 'G3': 55, 'A3': 57, 'B3': 59,
    'C4': 60, 'D4': 62, 'E4': 64, 'F4': 65, 'G4': 67, 'A4': 69, 'B4': 71,
    'C5': 72, 'D5': 74, 'E5': 76, 'F5': 77, 'G5': 79, 'A5': 81, 'B5': 83,
    'C6': 84, 'D6': 86, 'E6': 88, 'F6': 89, 'G6': 91, 'A6': 93, 'B6': 95,
    'C7': 96, 'D7': 98, 'E7': 100, 'F7': 101, 'G7': 103, 'A7': 105, 'B7': 107,
    'C8': 108, 'D8': 110, 'E8': 112, 'F8': 113, 'G8': 115, 'A8': 117, 'B8': 119,
    'C9': 120, 'D9': 122, 'E9': 124, 'F9': 125, 'G9': 127
}

def note_name_to_midi(note):
    return note_to_midi[note]

def create_chord(root, chord_type):
    chord = []
    if chord_type == 'maj':
        chord = [root, root + 'm', root + 'M']
    elif chord_type == 'min':
        chord = [root, root + 'm', root + 'M']
    elif chord_type == 'dim':
        chord = [root, root + 'dim', root + 'M']
    elif chord_type == 'aug':
        chord = [root, root + 'm', root + 'M']
    return chord

def generate_notes(scale, chord_types):
    notes = []
    for chord_type in chord_types:
        for note in scale:
            notes.extend(create_chord(note, chord_type))
    return notes

scale = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4']
chord_types = ['maj', 'min', 'dim', 'aug']
notes = generate_notes(scale, chord_types)

print(13.0)
print(14.0)
print(15.0)
print(16.0)

13.0
14.0
15.0
16.0


In [19]:
import mido

# Define the notes of the chord
notes = ['C0', 'E1', 'G2','A3','B4']

# Create a chord from the notes
chord = mido.MidiFile()
chord.ticks_per_beat = 480
track = mido.MidiTrack()
chord.tracks.append(track)

for note in notes:
    track.append(mido.Message('note_on', note=note_name_to_midi(note), velocity=100, time=0))
    track.append(mido.Message('note_off', note=note_name_to_midi(note), velocity=100, time=480))

# Save the chord to a MIDI file
chord.save('chord.mid')